In [3]:
config = {    
    'dataset_base_dir': '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SURREAL/train',
    'dataset_name': 'partial_0.8_5k_32_1',
    
    'fmap_direction': 'xy',
    'fmap_type': 'orig',
    'conditioning_types': {'evecs'},

} 

In [4]:
from my_code.datasets.surreal_cached_train_dataset import SurrealTrainDataset

dataset_train = SurrealTrainDataset(
    # f'data/SURREAL_full/full_datasets/{config["dataset_name"]}/train',
    f'{config["dataset_base_dir"]}/{config["dataset_name"]}/train',
    fmap_direction=config["fmap_direction"],
    fmap_input_type=config["fmap_type"],
    conditioning_types=config["conditioning_types"],
    mmap=True
    )

In [7]:
# dataset_train.fmaps, dataset_train.evecs_cond_first, dataset_train.evecs_cond_second 

# check if dataset_train.fmaps has NaN values
import torch

print('fmaps', torch.isnan(dataset_train.fmaps).any())
print('evecs_cond_first', torch.isnan(dataset_train.evecs_cond_first).any())
print('evecs_cond_second', torch.isnan(dataset_train.evecs_cond_second).any())

In [ ]:
# which elements in evecs_cond_second are NaN

nan_idx = torch.isnan(dataset_train.evecs_cond_second).nonzero()

print('evecs_cond_second', nan_idx)

# get the first indices of the elements in evecs_cond_second that are NaN

nan_idx_unique = torch.unique(nan_idx[:, 0])

print('evecs_cond_second', nan_idx_unique)

In [17]:
import os

def remove_nans(load_from, save_to):

    tensor_names = ['C_gt_xy', 'C_gt_yx', 'evals_first', 'evals_second', 'evecs_cond_first', 'evecs_cond_second']
    tensor_list = []
    
    for tensor_name in tensor_names:
        tensor_i = torch.load(f'{load_from}/{tensor_name}.pt')
        tensor_list.append(tensor_i)
    
    
    # get the first indices of the elements in each tensor that are NaN
    
    nan_idxs = []
    for tensor_i in tensor_list:
        nan_idx = torch.isnan(tensor_i).nonzero()
        nan_idx_unique = torch.unique(nan_idx[:, 0])
        
        if len(nan_idx_unique) > 0:
            for i in range(len(nan_idx_unique)):
                nan_idxs.append(nan_idx_unique[i].item())
        
    print('nan_idxs', len(nan_idxs), nan_idxs)
        
    if len(nan_idxs) == 0:
        print('No NaNs found')
        return
        
    idxs_normal = torch.tensor([i for i in range(len(tensor_list[0])) if i not in nan_idxs])
    
    print('idxs_normal', len(idxs_normal), idxs_normal)
    
    
    os.makedirs(save_to, exist_ok=True)
    for i in range(len(tensor_names)):
        tensor_name = tensor_names[i]
        tensor_i = tensor_list[i][idxs_normal]
        
        print(f"Saving {tensor_name} with shape {tensor_i.shape}")
        
        torch.save(tensor_i, f'{save_to}/{tensor_name}.pt')
        

remove_nans(load_from='/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SURREAL/train/partial_0.8_5k_32_1/train',
            save_to='/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SURREAL/train/partial_0.8_5k_32_1_fixed/train')  
    